# Divisão e preparação do dataset
Esse notebook é responsável por:
- Remover as colunas 'description' e 'url' (colunas que não são úteis para a previsão do preço)
- Dividir o dataset em treino, validação e teste

In [1]:
from Utils import *

## Removendo colunas 'description', 'url' e outras que avaliamos irrelevantes

Fizemos algumas análises nas colunas presentes no Dataset e concluimos que algumas delas podem mais ajudar 

#### Remover
- Region - 402 valores distintos
- condition - 7 valores distintos - 36% de undefined (Maior categoria) - só tem valores positivos (excellent, good, like new, fair, new, salvage, undefined)
- model - 24756 valores distintos - 0,75% undefined (4ª maior categoria)
- cylinders - 9 valores distintos -  31% de Undefined e other (1ª e 7ª maiores categorias) - (3,4,5,6,8,10,12, others, undefined)


#### Manter
- type - 14 valores distintos - 22% de undefined e other (3ª e 8ª maiores categorias) - SUV, sedan, undefined, pickup, truck, coupe, hatchback, other, wagon, van, convertible, mini-van, bus, offroad)
- title_status - 7 valores distintos - 0,37% Undefined (5ª maior categoria) - (clean, rebuilt, salvage, lien, undefined, missing, parts only)
- fuel - 6 valores distintos - 3,4% de undefined e other(5ª e 3ª maior categoria) - (gas, diesel, other, hybrid, undefined, eletric) 

In [3]:
df = pd.read_csv('../../Parte 1/datasets/clean_vehicles_2.csv')

In [4]:
df.loc[df.fuel == 'other', 'fuel'] = 'undefined'
df.loc[df.type == 'other', 'type'] = 'undefined'
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 316513 entries, 0 to 316512
Data columns (total 19 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   url           316513 non-null  object 
 1   region        316513 non-null  object 
 2   price         316513 non-null  int64  
 3   year          316513 non-null  int64  
 4   manufacturer  316513 non-null  object 
 5   model         316513 non-null  object 
 6   condition     316513 non-null  object 
 7   cylinders     316513 non-null  object 
 8   fuel          316513 non-null  object 
 9   odometer      316513 non-null  int64  
 10  title_status  316513 non-null  object 
 11  transmission  316513 non-null  object 
 12  drive         316513 non-null  object 
 13  type          316513 non-null  object 
 14  paint_color   316513 non-null  object 
 15  description   316513 non-null  object 
 16  state         316513 non-null  object 
 17  lat           316513 non-null  float64
 18  long

In [5]:
df.drop(columns=['url', 'description','paint_color','lat','long', 'region', 'condition', 'model', 'cylinders'], inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 316513 entries, 0 to 316512
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   price         316513 non-null  int64 
 1   year          316513 non-null  int64 
 2   manufacturer  316513 non-null  object
 3   fuel          316513 non-null  object
 4   odometer      316513 non-null  int64 
 5   title_status  316513 non-null  object
 6   transmission  316513 non-null  object
 7   drive         316513 non-null  object
 8   type          316513 non-null  object
 9   state         316513 non-null  object
dtypes: int64(3), object(7)
memory usage: 24.1+ MB


## Adicionando categorias de preço

- very low price: 0 - 3.000
- low price: 3.000 - 6.000
- low average price: 6.000 - 10.000
- average price: 10.000 - 15.000
- high average price: 15.000 - 20.000
- high price: 20.000 - 30.000
- very high price: 30.000 - 120.000

As categorias foram definidas através de algumas pesquisas sobre o poder aquisitivo de um cidadão americano.

In [7]:
def categoryGen(x):
    if x < 3000:
        return "very low price"
    elif x < 6000:
        return "low price"
    elif x < 10000:
        return "average low price"
    elif x < 15000:
        return "average price"
    elif x < 20000:
        return "average high price"
    elif x  < 30000:
        return "high price"
    else:
        return "very high price"


df['pricecategory'] = df['price'].apply(categoryGen)
df['pricecategory'].value_counts()


average low price     66571
average price         63155
low price             55272
average high price    46782
high price            45286
very high price       23013
very low price        16434
Name: pricecategory, dtype: int64

In [9]:
df.to_csv('../datasets/final_vehicles.csv')

## Divisão do dataset em treino, validação e teste

In [11]:
from sklearn.model_selection import train_test_split

In [13]:
x_df = df.drop(columns=['price'])
target = df['price']
xTrain, xTest, yTrain, yTest = train_test_split(x_df, target, random_state=1)
xTrain, xValidation, yTrain, yValidation = train_test_split(xTrain, yTrain, random_state=1)

In [15]:
train_df = xTrain
train_df['price'] = yTrain
validation_df = xValidation
validation_df['price'] =yValidation
test_df = xTest
test_df['price'] = yTest

/Users/matheus/miniconda3/envs/geo-env/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Dividimos os datasets que serão usados para os classificadores e para os regressores em duas pastas:
- 'classifier' para os que serão usados nos classificadores
- 'regressor' para os que serão usados nos regressores

In [22]:
# Definindo os datasets que serão usados para treinar, validar e testar os regressores
train_df_regressor = train_df.drop(columns=['pricecategory'])
validation_df_regressor = validation_df.drop(columns=['pricecategory'])
test_df_regressor = test_df.drop(columns=['pricecategory'])

# Datasets que serão usados para os classificadores
train_df_classifier = train_df.drop(columns=['price'])
validation_df_classifier = validation_df.drop(columns=['price'])
test_df_classifier = test_df.drop(columns=['price'])

In [31]:
print("Tamanho do dataset de treino: %d amostras, %.2f%% do dataset completo." % (len(train_df), len(train_df)/len(df)*100))
print("Tamanho do dataset de validação: %d amostras, %.2f%% do dataset completo." % (len(validation_df), len(validation_df)/len(df)*100))
print("Tamanho do dataset de teste: %d amostras, %.2f%% do dataset completo." % (len(test_df), len(test_df)/len(df)*100))

Tamanho do dataset de treino: 178038 amostras, 56.25% do dataset completo.
Tamanho do dataset de validação: 59346 amostras, 18.75% do dataset completo.
Tamanho do dataset de teste: 79129 amostras, 25.00% do dataset completo.


In [32]:
# Salvando os datasets de treino, validação e testes para os regressores e classificadores
train_df_regressor.to_csv('../datasets/regressor/train_regressor_vehicles.csv', index=False)
validation_df_regressor.to_csv('../datasets/regressor/validation_regressor_vehicles.csv', index=False)
test_df_regressor.to_csv('../datasets/regressor/test_regressor_vehicles.csv', index=False)

train_df_classifier.to_csv('../datasets/classifier/train_classifier_vehicles.csv', index=False)
validation_df_classifier.to_csv('../datasets/classifier/validation_classifier_vehicles.csv', index=False)
test_df_classifier.to_csv('../datasets/classifier/test_classifier_vehicles.csv', index=False)